In [ ]:
 import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from google.colab import drive
from zipfile import ZipFile 
from sklearn.model_selection import train_test_split
import cv2

import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# clear cuda cache
torch.cuda.empty_cache()
!nvidia-smi

Thu Dec 17 20:27:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    39W / 250W |   3401MiB / 16280MiB |    100%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# manually kill process that takes gpu memory 
!kill 1075
!ps -elf | grep python

/bin/bash: line 0: kill: (1075) - No such process
4 S root          57       1  0  80   0 - 49474 epoll_ 14:24 ?        00:01:08 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --LargeFileManager.delete_to_trash=False --MappingKernelManager.root_dir="/content"
4 S root         573       1  0  80   0 -  4596 wait   14:52 ?        00:00:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
4 S root         575     573  0  80   0 -  7334 pipe_r 14:52 ?        00:00:00 python3 /opt/google/drive/drive-filter.py
4 S root        1438      57 81  80   0 - 5657655 select 17:40 ?      01:06:39 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-823c607c-4bd8-4082-a930-1de6158d1465.json
0 S root        1619    1438  0  80   0 -  9800 wait   19:02 ?        00:00:00 /bin/bash -c ps -elf |

In [ ]:
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.54.1)


In [ ]:
# connect with google drive and unzip the zipped image folder
drive.mount('/content/gdrive')

train_zip_path = "/content/gdrive/MyDrive/kaggle-competition-datasets/cassava-leaf-disease-classification/train_images.zip"
with ZipFile(train_zip_path, 'r') as zip_f: 
  zip_f.extractall(path='/content') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# load leaf images dataset
all_images_id = pd.read_csv('/content/train.csv')['image_id'].to_numpy()
all_labels = pd.read_csv('/content/train.csv')['label'].to_numpy()

In [ ]:
# define the image-read fucntion
def read_img_from_path(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1].copy()
    #print(im_rgb)
    return im_rgb

test_img = read_img_from_path('/content/train_images/1000015157.jpg')
print(test_img.shape)

(600, 800, 3)


In [ ]:
# define image transformation functions
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465),
                          (0.2023, 0.1994, 0.2010))
    ])

In [ ]:
# split images as validation set and training set by the ratio of 8 : 2
n = len(all_images_id)
train_image_id, val_image_id, train_image_label, \
  val_image_label = train_test_split(all_images_id, all_labels, test_size = 0.2)

In [ ]:
# define the training and validaiton dataset
class TrainDataset(Dataset):
  """ Leaves Training Dataset """
  def __init__(self, train_img_id, train_img_label, transform = None):
    self.train_img_id = train_img_id
    self.train_img_label = train_img_label
    self.transform = transform
  
  def __len__(self):
    return len(self.train_img_id)

  def __getitem__(self, idx): 
    img_path_prefix = '/content/train_images/'
    img_path = img_path_prefix + str(self.train_img_id[idx])
    img = read_img_from_path(img_path)
    label = self.train_img_label[idx]
    if self.transform:
      img = self.transform(img)
    return (img, label)

class ValDataset(Dataset):
  """ Leaves Validation Dataset """
  def __init__(self, val_img_id, val_img_label):
    self.val_img_id = val_img_id
    self.val_img_label = val_img_label
    self.transform = transform
  
  def __len__(self):
    return len(self.val_img_id)

  def __getitem__(self, idx): 
    img_path_prefix = '/content/train_images/'
    img_path = img_path_prefix + str(self.val_img_id[idx])
    img = read_img_from_path(img_path)
    if self.transform:
      img = self.transform(img)    
    label = self.val_img_label[idx]
    return (img, label)

In [ ]:
# define the training and validation data loader
batch_size = 16
train_dataset = TrainDataset(train_image_id, train_image_label, transform)
val_dataset = ValDataset(val_image_id, val_image_label)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [ ]:
# load a pretrained res50 network and modify its last linear layer to 
# match the number of outputs with this problem 
num_category = 5
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(2048, num_category)
resnet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# define a loss function and optimizer
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=lr, momentum=0.9)

In [ ]:
# train the network
num_epochs = 3
print('Start training...')
for epoch in range(num_epochs):
  running_loss = 0.0
  cnt = 0
  # train process
  for data in tqdm(train_dataloader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = resnet50(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    cnt += 1
    if cnt % 59 == 0:
      print('Training [%d, %5d] loss: %.3f' % (epoch + 1, cnt + 1, running_loss / 60))
      running_loss = 0.0
    
  # validaiton process
  running_loss = 0.0
  correct_prediction_cnt = 0
  batch_cnt = 0
  with torch.no_grad():
    for data in tqdm(val_dataloader):
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      outputs = resnet50(inputs)
      loss = criterion(outputs, labels)

      running_loss += loss
      correct_prediction_cnt += torch.eq(torch.argmax(outputs, dim=1), labels).sum().item()
      batch_cnt += 1 
    print('Validation [%d] loss: %.3f; accuracy: %.3f' % (epoch, running_loss / batch_cnt, \
        correct_prediction_cnt * 1.0 / len(val_dataloader.dataset)))


Start training...


Training [1,    60] loss: 1.038
Training [1,   119] loss: 0.794
Training [1,   178] loss: 0.691
Training [1,   237] loss: 0.678
Training [1,   296] loss: 0.598
Training [1,   355] loss: 0.548
Training [1,   414] loss: 0.587
Training [1,   473] loss: 0.538
Training [1,   532] loss: 0.455
Training [1,   591] loss: 0.527
Training [1,   650] loss: 0.472
Training [1,   709] loss: 0.484
Training [1,   768] loss: 0.470
Training [1,   827] loss: 0.456
Training [1,   886] loss: 0.455
Training [1,   945] loss: 0.468
Training [1,  1004] loss: 0.442
Training [1,  1063] loss: 0.482




Validation [0] loss: 0.457; accuracy: 0.843


Training [2,    60] loss: 0.411
Training [2,   119] loss: 0.369
Training [2,   178] loss: 0.380
Training [2,   237] loss: 0.361
Training [2,   296] loss: 0.429
Training [2,   355] loss: 0.380
Training [2,   414] loss: 0.384
Training [2,   473] loss: 0.360
Training [2,   532] loss: 0.355
Training [2,   591] loss: 0.389
Training [2,   650] loss: 0.354
Training [2,   709] loss: 0.346
Training [2,   768] loss: 0.421
Training [2,   827] loss: 0.383
Training [2,   886] loss: 0.370
Training [2,   945] loss: 0.360
Training [2,  1004] loss: 0.404
Training [2,  1063] loss: 0.375




Validation [1] loss: 0.407; accuracy: 0.868


Training [3,    60] loss: 0.251
Training [3,   119] loss: 0.252
Training [3,   178] loss: 0.279
Training [3,   237] loss: 0.295
Training [3,   296] loss: 0.309
Training [3,   355] loss: 0.292
Training [3,   414] loss: 0.297
Training [3,   473] loss: 0.244
Training [3,   532] loss: 0.336
Training [3,   591] loss: 0.303
Training [3,   650] loss: 0.282
Training [3,   709] loss: 0.281
Training [3,   768] loss: 0.304
Training [3,   827] loss: 0.262
Training [3,   886] loss: 0.280
Training [3,   945] loss: 0.290
Training [3,  1004] loss: 0.303
Training [3,  1063] loss: 0.311




Validation [2] loss: 0.404; accuracy: 0.868


In [ ]:
# save the trained model
model_state_save_path = '/content/trained_epoch_3_dict_model.pt'
torch.save(resnet50.state_dict(), model_state_save_path)

In [ ]:
# load the trained resnet50 model
pretrained_model_path = '/content/trained_epoch_3_dict_model.pt'
resnet50_copy = torchvision.models.resnet50(pretrained=False)
resnet50_copy.fc = nn.Linear(2048, num_category)
resnet50_copy.load_state_dict(torch.load(pretrained_model_path))

<All keys matched successfully>

In [ ]:
# load the test images id
test_img_path = '/content/test_images'
test_img_id = list(os.listdir(test_img_path))
print(test_img_id)

# create the test dataset
class TestDataset(Dataset):
  """ Leaves Test Dataset """
  def __init__(self, test_img_id, transform = None):
    self.test_img_id = test_img_id
    self.transform = transform
  
  def __len__(self):
    return len(self.test_img_id)

  def __getitem__(self, idx): 
    img_path_prefix = '/content/test_images/'
    img_path = img_path_prefix + str(self.test_img_id[idx])
    img = read_img_from_path(img_path)
    if self.transform:
      img = self.transform(img)
    return img, self.test_img_id[idx]

# define the test dataloader
test_dataset = TestDataset(test_img_id, transform)
test_dataloader = DataLoader(test_dataset, batch_size = 1, shuffle = False, num_workers = 0)

['2216849948.jpg']


In [ ]:
# infer the labels of the test dataset
res_test_pred_label = []
res_test_img_id = []
with torch.no_grad():
  for tst_img, tst_img_filename in test_dataloader:
    pred = resnet50_copy(tst_img)
    pred_label = torch.argmax(pred, dim=1).numpy()[0]
    tst_img_id = tst_img_filename[0][:-4]
    res_test_img_id.append(tst_img_id)
    res_test_pred_label.append(pred_label)
    
# convert the submission to csv
output_path = '/content/submission.csv'
column_header = ['image_id', 'label']
submission = pd.DataFrame(zip(res_test_img_id, res_test_pred_label), columns=column_header)
submission.to_csv(path_or_buf = output_path, index = False)